### Import packages

Import pandas to perform data pre-processing

In [1]:
import pandas as pd

### Read in dataset into dataframe
Just need to add the dataset into the "datasets" directory. A Google Drive link to the original "Alzheimer's Disease and Healthy Aging Data" dataset is included in the "Code and Documentation" section of the final report.

This code is performing some basic exploratory data analysis to understand our dataset better.

In [2]:
# Read csv dataset into pandas dataframe
df = pd.read_csv("datasets/Alzheimer_s_Disease_and_Healthy_Aging_Data_20240418.csv")
pd.set_option('display.max_columns', None)

In [3]:
# Print out all columns available in the dataset
df_keys = df.keys()
print("Number of keys: %s" % len(df_keys))
print(df_keys)

Number of keys: 31
Index(['RowId', 'YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc',
       'Datasource', 'Class', 'Topic', 'Question', 'Data_Value_Unit',
       'DataValueTypeID', 'Data_Value_Type', 'Data_Value', 'Data_Value_Alt',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'Geolocation', 'ClassID', 'TopicID', 'QuestionID',
       'LocationID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2'],
      dtype='object')


In [4]:
# Explore what questions are included in the survey dataset, which will become our treatment and outcome variables
unique_questions = df["Question"].unique()
print("Number of unique questions: %s" % len(unique_questions))
print(df["Question"].value_counts())

Number of unique questions: 39
Percentage of older adults who are experiencing frequent mental distress                                                                                                     11092
Percentage of older adults who have not had any leisure time physical activity in the past month                                                                             11092
Physically unhealthy days (mean number of days in past month)                                                                                                                11092
Percentage of older adults who reported influenza vaccine within the past year                                                                                               11092
Percentage of older adults who are currently obese, with a body mass index (BMI) of 30 or more                                                                               11092
Percentage of older adults who have smoked at least 100 cigarettes in thei

In [5]:
# Explore stratifications in the dataset, which will become our confounding variables
# StratificationCategory1 is just "Age Group"
unique_stratcat1 = df["StratificationCategory1"].unique()
print("Number of unique StratificationCategory1: %s" % len(unique_stratcat1))

unique_strat1 = df["Stratification1"].unique()
print("Number of unique Age Groups: %s" % len(unique_strat1))
print(df["Stratification1"].value_counts())


Number of unique StratificationCategory1: 1
Number of unique Age Groups: 3
Overall              95233
50-64 years          94741
65 years or older    94168
Name: Stratification1, dtype: int64


In [6]:
# Explore stratifications in the dataset, which will become our confounding variables
# StratificationCategory2 is either "Race/Ethnicity" or "Gender"
unique_stratcat2 = df["StratificationCategory2"].unique()
print("Number of unique StratificationCategory2: %s" % len(unique_stratcat2))
print(df["StratificationCategory2"].value_counts())
print()

unique_strat2 = df["Stratification2"].unique()
print("Number of unique Age Groups: %s" % len(unique_strat2))
print(df["Stratification2"].value_counts())

Number of unique StratificationCategory2: 3
Race/Ethnicity    178431
Gender             68838
Name: StratificationCategory2, dtype: int64

Number of unique Age Groups: 8
White, non-Hispanic         36450
Hispanic                    36326
Black, non-Hispanic         35770
Native Am/Alaskan Native    35021
Asian/Pacific Islander      34864
Female                      34596
Male                        34242
Name: Stratification2, dtype: int64


### Format dataframe for causal analysis
Now that we've performed some basic EDA, we want to format the dataset so that we can perform causal analysis on it. 

Create columns for confounders, treatment, and outcome

In [49]:
df.keys()

Index(['RowId', 'YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc',
       'Datasource', 'Class', 'Topic', 'Question', 'Data_Value_Unit',
       'DataValueTypeID', 'Data_Value_Type', 'Data_Value', 'Data_Value_Alt',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'Geolocation', 'ClassID', 'TopicID', 'QuestionID',
       'LocationID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2'],
      dtype='object')

We extract separate dataframes for our treatment variables of interest and our outcome variables of interest, as "df_treatments" and "df_outcomes" respectively. We will later merge these two into a full dataset for causal analysis.

In [50]:
treatments = ["Eating 3 or more vegetables daily",
              "Prevalence of sufficient sleep"]
outcomes = ["Frequent mental distress",
            "Lifetime diagnosis of depression"]

# Treatments df with original shape (11084, 31)
df_treatments = df[df["Topic"].isin(treatments)]
# Drop irrelevant columns
df_treatments = df_treatments.drop(columns=["RowId", "YearStart", "YearEnd", "Datasource", "Class", "Question", "LocationDesc", "Data_Value_Unit", "Data_Value_Alt", 
                                            "DataValueTypeID", "Data_Value_Type", "Data_Value_Footnote_Symbol", "Data_Value_Footnote", "StratificationCategory1", 
                                            "Stratification1", "StratificationCategory2", "Stratification2", "Geolocation", "ClassID", "TopicID", "QuestionID", 
                                            "LocationID", "StratificationCategoryID1", "StratificationCategoryID2"])
# Rename columns to more intuitive names
df_treatments.rename(columns={"LocationAbbr": "Location", "Topic": "Treatment", "Data_Value": "Treatment_Value", "Low_Confidence_Limit": "Treatment_Low_Limit", 
                              "High_Confidence_Limit": "Treatment_High_Limit", "StratificationID1": "Age", "StratificationID2": "Gender/Race"}, inplace = True)
df_treatments.replace("Eating 3 or more vegetables daily", "Vegetables", inplace = True)
df_treatments.replace("Prevalence of sufficient sleep", "Sleep", inplace = True)
# Drop rows with at least one NaN value
df_treatments.dropna(inplace = True)
# Treatments df final shape: (7663, 7)

# Outcomes df with original shape (22184, 31)
df_outcomes = df[df["Topic"].isin(outcomes)]
df_outcomes = df_outcomes.drop(columns=["RowId", "YearStart", "YearEnd", "Datasource", "Class", "Question", "LocationDesc", "Data_Value_Unit", "Data_Value_Alt", 
                                            "DataValueTypeID", "Data_Value_Type", "Data_Value_Footnote_Symbol", "Data_Value_Footnote", "StratificationCategory1", 
                                            "Stratification1", "StratificationCategory2", "Stratification2", "Geolocation", "ClassID", "TopicID", "QuestionID", 
                                            "LocationID", "StratificationCategoryID1", "StratificationCategoryID2"])
df_outcomes.rename(columns={"LocationAbbr": "Location", "Topic": "Outcome", "Data_Value": "Outcome_Value", "Low_Confidence_Limit": "Outcome_Low_Limit", 
                            "High_Confidence_Limit": "Outcome_High_Limit", "StratificationID1": "Age", "StratificationID2": "Gender/Race"}, inplace = True)
df_outcomes.replace("Frequent mental distress", "Distress", inplace = True)
df_outcomes.replace("Lifetime diagnosis of depression", "Depression", inplace = True)
df_outcomes.dropna(inplace = True)
# Treatments df final shape: (14508, 7)

df_treatments.keys()

Index(['Location', 'Treatment', 'Treatment_Value', 'Treatment_Low_Limit',
       'Treatment_High_Limit', 'Age', 'Gender/Race'],
      dtype='object')

In [51]:
df_treatments

,Location,Treatment,Treatment_Value,Treatment_Low_Limit,Treatment_High_Limit,Age,Gender/Race
18,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL
27,WEST,Sleep,63.7,62.1,65.2,5064,OVERALL
34,OH,Sleep,60.1,58.0,62.2,5064,OVERALL
41,WEST,Sleep,61.0,53.0,68.5,AGE_OVERALL,NAA
42,WY,Sleep,66.9,63.9,69.8,5064,OVERALL
...,...,...,...,...,...,...,...
283635,WI,Vegetables,8.9,7.2,11.0,AGE_OVERALL,MALE
283642,WV,Sleep,62.4,60.2,64.5,AGE_OVERALL,FEMALE
283678,WY,Vegetables,14.6,12.4,17.1,5064,WHT
283770,WV,Vegetables,15.3,12.7,18.3,65PLUS,FEMALE


In [52]:
df_outcomes

,Location,Outcome,Outcome_Value,Outcome_Low_Limit,Outcome_High_Limit,Age,Gender/Race
0,WEST,Distress,15.4,13.9,17.1,5064,FEMALE
1,NRE,Distress,10.3,9.3,11.5,5064,MALE
2,MDW,Distress,10.9,8.8,13.6,AGE_OVERALL,HIS
3,SOU,Depression,18.4,17.4,19.5,65PLUS,FEMALE
8,ID,Depression,16.9,15.0,19.1,AGE_OVERALL,OVERALL
...,...,...,...,...,...,...,...
284093,WY,Depression,16.2,14.2,18.3,65PLUS,OVERALL
284104,WV,Distress,23.9,13.6,38.6,5064,BLK
284111,WI,Depression,12.2,10.5,14.2,65PLUS,WHT
284118,WV,Depression,16.0,13.3,19.2,65PLUS,MALE


To create the final dataset, we match samples based on our confounding variables of demographic background ("Location", "Age", "Gender/Race").

In [53]:
# Find rows with matching values in both treatments and outcomes df
df_treatments_outcomes = pd.merge(df_treatments, df_outcomes, on=["Location", "Age", "Gender/Race"])
df_treatments_outcomes

,Location,Treatment,Treatment_Value,Treatment_Low_Limit,Treatment_High_Limit,Age,Gender/Race,Outcome,Outcome_Value,Outcome_Low_Limit,Outcome_High_Limit
0,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL,Distress,8.4,6.3,11.0
1,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL,Depression,12.8,10.5,15.4
2,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL,Distress,7.6,6.0,9.5
3,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL,Depression,16.0,13.8,18.5
4,MS,Sleep,72.1,69.2,74.7,65PLUS,OVERALL,Distress,8.6,7.2,10.3
...,...,...,...,...,...,...,...,...,...,...,...
111780,VA,Sleep,63.3,46.1,77.6,AGE_OVERALL,NAA,Distress,28.3,15.1,46.5
111781,WV,Sleep,43.1,30.1,57.1,5064,BLK,Depression,27.5,17.0,41.4
111782,WV,Sleep,43.1,30.1,57.1,5064,BLK,Depression,24.2,13.3,39.9
111783,WV,Sleep,43.1,30.1,57.1,5064,BLK,Distress,33.0,20.4,48.7


In [54]:
# Save to output dataframe
"""
df_treatments_outcomes dataset
    shape: (111785, 11)
"""
df_treatments_outcomes.to_csv("datasets/df_treatments_outcomes.csv", index=False)